In [ ]:
import tushare as ts
# ts.set_token('bcdcd7446a8a71251e4d8d0fc6d4709400a27384ae370a879dcfffd3')
ts.set_token('dfb6e9f4f9a3db86c59a3a0f680a9bdc46ed1b5adbf1e354c7faa761')
pro = ts.pro_api()

# 一、获取基金数据
1.从fund_basic获取场内基金基础信息
2.按照1中的代码，从fund_share获取基金规模，并按照规模排序
3.按照生成的基金规模排序表，手动选择不同品类的ETF，预计选择30+
4.从fund_daily获取ETF成立至今的行情

# 二、实现一个demo
1.测试talib库，使用移动平均线
2.选择510300.SH沪深300ETF，根据均线形成信号，形成trade_info仓位权重，
3.使用backtrader进行回测

# 三、策略规范化
1.使用组合，构建时序规则模型
2.品种风险衡量，如相关性
3.资金分配，比如风险平价
4.设置止损/止盈条件，综合考虑成本/技术指标/趋势
5.考虑使用因子投资手册方法论分析因子

# 四、策略优化
1.尝试使用期货/期指构建CTA时序规则或截面模型